This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace.

Step 1: Setup
Simply install the provided requirements.txt file by running the following command:
```bash
pip install -r requirements.txt
```

In [242]:
import os
def set_open_ai_api():
    os.environ["OPENAI_API_KEY"] = # api key removed afger runningcell
    os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

You can now directly skip to step 4 since the fake listings have been saved for you and since each new run of the cells of step 2&3 would generate a new set of listings (reproducibility!).

Step 2 & 3: Generating Fake Listings Using LLM and Saving Them

In [ ]:
import openai
import os

from langchain.memory import ConversationBufferMemory
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

from langchain.llms import OpenAI


In [ ]:
from langchain.prompts import PromptTemplate

listing_template = PromptTemplate(template=
"""Neighborhood: {neighborhood}
Price: ${price}
Bedrooms: {bathrooms}
Bathrooms: {bedrooms}
House Size: {house_size} sqft

Description: {description}

Neighborhood Description: {neighborhood_description}
""",
                                  input_variables=["neighborhood", "price", "bedrooms", "bathrooms",
                                                   "house_size", "description", "neighborhood_description"]
                                  )



In [ ]:
example_inputs = [
    {
        "neighborhood": "Downtown Abbey",
        "price": 300000,
        "bedrooms": 3,
        "bathrooms": 2,
        "house_size": 2000,
        "description": "A beautiful appartment with a large terrace overlooking the city's skyscrapers with 3 well-lit bedrooms and 2 bathrooms means you have enough room even for the occasional guest coming over to celebrate new year's with you :)",
        "neighborhood_description": "A vibrant neighborhood which is very centrally located and specially well connected public transport connections means you are close to everything you need for your everyday life from groceries to cafes and restaurants and shopping."
    },
    {
        "neighborhood": "City Heights",
        "price": 500000,
        "bedrooms": 4,
        "bathrooms": 3,
        "house_size": 3000,
        "description": "A cozy house with a wonderful garden overlooking the city from its hillside location",
        "neighborhood_description": "Located in the city outskirts, this neighborhood is known for its great views and quiet streets with a very family-friendly atmosphere. In no time you are in the midst of a beautiful forest where you can calm all your senses and breath in the cool fresh air. Sightings of deer and other wildlife are common."
    },
    {
        "neighborhood": "Green Oaks",
        "price": 800000,
        "bedrooms": 3,
        "bathrooms": 2,
        "house_size": 2000,
        "description": "A luxurious villa with a large garden and a swimming pool with a winter garden, 2 parking spots, and a garage as well as a large terrace and wonderful finishing.",
        "neighborhood_description": "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
    }
]

In [ ]:
from langchain.prompts import FewShotPromptTemplate

few_shot_template = FewShotPromptTemplate(
    example_prompt=listing_template,
    input_variables=["input"],
    examples=example_inputs,
    suffix="Use the examples above to generate the following: {input}",
)

listing_seperator = "===="
instruction = "Generate {} listings of houses. Be creative in regards neighborhood_description, and description and keep it confined to three sentences each. Keep the prices between $125000 $1500000, the number of bedrooms under 6 and the number of bathrooms under 3 and the total area under 3500sqft. VERY IMPORTANT: Output the results in the same format as the examples keeping even the order of properties the same as in the examples. Add the characters {} before each new listing"
num_listings = 30
prompt_to_use = few_shot_template.format(input=instruction.format(num_listings, listing_seperator))



In [ ]:
print(prompt_to_use)

In [ ]:
from langchain.chat_models import ChatOpenAI

model_name = "gpt-3.5-turbo"
llm = OpenAI(model_name=model_name, temperature=0.0, max_tokens=3500)

In [ ]:
result = llm(prompt=prompt_to_use)

In [ ]:
print(f"LLM result:\n{result}")

In [ ]:
import pandas as pd
import locale
import re

# Set the locale to 'en_US.UTF-8' for parsing numbers with commas
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Split the result into individual listings
listings = result.split(listing_seperator)[1:]
# print(f"Here are the listings in a list:\n{listings}")

# Define the columns based on input_variables
columns = ["neighborhood", "price", "bedrooms", "bathrooms", "house_size", "description", "neighborhood_description"]

# Parse the listings into a list of dictionaries
data = []
for listing in listings:
    print(f"Parsing listing: {listing[1:150]} ...\n")
    try:
        entry = {
            "neighborhood": re.search(r"^Neighborhood: (.+)$", listing, re.MULTILINE).group(1),
            "price": locale.atoi(re.search(r"^Price: \$(.+)$", listing, re.MULTILINE).group(1)),
            "bedrooms": int(re.search(r"^Bedrooms: (.+)$", listing, re.MULTILINE).group(1)),
            "bathrooms": int(re.search(r"^Bathrooms: (.+)$", listing, re.MULTILINE).group(1)),
            "house_size": locale.atoi(re.search(r"^House Size: (.+) sqft$", listing, re.MULTILINE).group(1)),
            "description": re.search(r"^Description: (.+)$", listing, re.MULTILINE).group(1),
            "neighborhood_description": re.search(r"^Neighborhood Description: (.+)$", listing, re.MULTILINE).group(1)
        }

        data.append(entry)
    except Exception as e:
        print("Error parsing listing")
        print(listing)
        print(f"Because of \n{e}")
        print("\n")
        continue

# Create a DataFrame
df = pd.DataFrame(data, columns=columns)

Let's view the dataframe and assert the generated input makes sense and finally save into a csv.

In [ ]:
df.head(n=num_listings)

In [ ]:
df.to_csv("listings.csv", index=False)


Step 4: Building the User Preference Interface

In [302]:
def ask_llm_and_print_question_and_response(chain,question):
    response=chain(inputs={"question": ("%s" % question)})
    print(f"The user asked: {question}")
    print(f"The smart assistant answered: {response['answer']}")
    return response

In [284]:
from langchain.llms import OpenAIChat
set_open_ai_api()
llm = OpenAIChat(temperature=0.0)

/home/tumaaa/work/repos/udacity_nano_degree_generative_ai/.venv/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [285]:
# Hard-coding the user preferences using the ones provided in the project - with minor modifications - for reproducibility purposes wit

answers = [
    "A comfortable house with at least 3 bedrooms with a spacious kitchen and a cozy living room with at least 2000sqft",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "$500,000",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
    ]
questions = [
    "How big should your house be?",
    "What are 3 most important things for you in choosing this property?",
    "What is the maximum price you would be willing to pay?"
    "Which amenities would you like?",
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",
]

We create a conversation chain with memory and simulate the question - answer interaction such that we feed its memory with the questions and answers since running the interaction for real might not necessarily lead the llm to asking the right questions due to its unpredictability

In [290]:
from langchain.memory import ChatMessageHistory
def create_chat_message_history(questions, answers):
    chat_message_history=ChatMessageHistory()
    for question, answer in zip(questions, answers):
        chat_message_history.add_ai_message(question)
        chat_message_history.add_user_message(answer)
    return chat_message_history

In [291]:
base_system_prompt = """You are a real estate agent. You have a new client who is looking for a new home.

The client has provided answers to important questions in the context.

Proceed as follows:
- First ask the user if he/she is ready to receive recommendations.
    - If he/she enters says 'yes', summarize your understanding of the user preferences and ask the user if he/she is ready to receive recommendations telling the user to either type 'yes' or 'no'
        - If he types 'yes' you match the user preferences with listings and make suggestions."""

In [303]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

memory=ConversationBufferMemory(conversation_history=create_chat_message_history(questions,answers), llm=llm, chat_memory=create_chat_message_history(questions,answers),memory_key="context")

qa_template = PromptTemplate(template=
                             ("""%s

Context: {context}

Question: {question}
""" % base_system_prompt),
                             input_variables=["context", "question"]
                             )
qa_chain = ConversationChain(llm=llm,  memory=memory, prompt=qa_template,input_key="question",output_key="answer", verbose=False)


In [304]:
question = "I am waiting for your recommendations"
ask_llm_and_print_question_and_response(qa_chain,question=question)

The user asked: I am waiting for your recommendations
The smart assistant answered: Are you ready to receive recommendations based on your preferences? If so, please type 'yes' or 'no'.


{'question': 'I am waiting for your recommendations',
 'context': 'AI: How big should your house be?\nHuman: A comfortable house with at least 3 bedrooms with a spacious kitchen and a cozy living room with at least 2000sqft\nAI: What are 3 most important things for you in choosing this property?\nHuman: A quiet neighborhood, good local schools, and convenient shopping options.\nAI: What is the maximum price you would be willing to pay?Which amenities would you like?\nHuman: $500,000\nAI: Which transportation options are important to you?\nHuman: A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.\nAI: How urban do you want your neighborhood to be?\nHuman: Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.',
 'answer': "Are you ready to receive recommendations based on your preferences? If so, please type 'yes' or 'no'."}

In [305]:
question = "yes"
response=ask_llm_and_print_question_and_response(qa_chain,question=question)

The user asked: yes
The smart assistant answered: Based on your preferences, I understand that you are looking for a comfortable house with at least 3 bedrooms, a spacious kitchen, and a cozy living room with at least 2000sqft. You value a quiet neighborhood, good local schools, and convenient shopping options. Your maximum budget is $500,000 and you are looking for a property with a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. You also prefer easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. 

Are you ready to receive recommendations based on these preferences? If so, please type 'yes' or 'no'.


In [306]:
question = "yes"
response=ask_llm_and_print_question_and_response(qa_chain,question=question)

The user asked: yes
The smart assistant answered: Great! Based on your preferences, I have found a few listings that match what you are looking for. Here are some recommendations:

1. 3-bedroom, 2.5-bathroom house with a spacious kitchen, cozy living room, and 2200sqft of living space in a quiet neighborhood with top-rated schools nearby. This property features a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. It is also conveniently located near a reliable bus line, major highway, and bike-friendly roads. Price: $480,000

2. 4-bedroom, 2-bathroom house with a large kitchen, inviting living room, and 2100sqft of living space in a peaceful neighborhood with excellent schools and shopping options. This property includes a backyard for gardening, a two-car garage, and a modern heating system. It is situated close to a bus line, major highway, and bike-friendly roads. Price: $495,000

3. 3-bedroom, 2.5-bathroom house with a modern kitchen, comfortab

We can see the result above is totally fake. Let's see what we can do about this using our generated listings and vector db'

Step 5: Using the Generated Listings to Provide Real Context

In [307]:
from typing import List, Dict, Any
from langchain.memory import ConversationBufferMemory

# there is a solution to ConversationChain problem of not being able to use more than one input:
# The ConversationBufferMemory is extended to be able to pass other information.
# Thanks to https://github.com/langchain-ai/langchain/issues/1800#issuecomment-1598128244
class ExtendedConversationBufferMemory(ConversationBufferMemory):
    extra_variables:List[str] = []

    @property
    def memory_variables(self) -> List[str]:
        """Will always return list of memory variables."""
        return [self.memory_key] + self.extra_variables

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Return buffer with history and extra variables"""
        d = super().load_memory_variables(inputs)
        d.update({k:inputs.get(k) for k in self.extra_variables})
        return d

In [313]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate


memory_with_listings=ExtendedConversationBufferMemory(
    conversation_history=create_chat_message_history(questions,answers),
    llm=llm,
    chat_memory=create_chat_message_history(questions,answers),
    memory_key="context",
    extra_variables=["listings"])


qa_template_with_listings=PromptTemplate(template=
                          (("""%s. Only provide suggestions from the available listings. '

Context: {context}

Listings: {listings}

Question: {question}
""") % base_system_prompt),
                          input_variables=["context", "listings", "question"]
                          )



qa_chain_with_listings = ConversationChain(llm=llm,  memory=memory_with_listings, prompt=qa_template_with_listings,input_key="question" ,output_key="answer", verbose=False)



In [314]:
response=ask_llm_and_print_question_and_response(qa_chain_with_listings,question="I am waiting for your recommendations")
preference_summary_by_smart_assistant=response['answer']

The user asked: I am waiting for your recommendations
The smart assistant answered: First, are you ready to receive recommendations based on your preferences? If so, I understand that you are looking for a comfortable house with at least 3 bedrooms, a spacious kitchen, and a cozy living room with at least 2000sqft. Your top priorities are a quiet neighborhood, good local schools, and convenient shopping options. Your maximum budget is $500,000 and you are looking for a property with a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Additionally, you prefer easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. Are you ready to receive recommendations? Please type 'yes' or 'no'.


Now we use the preference summary by the smart assistant to retreive suitable listings from the vector db

In [315]:
from langchain.document_loaders import CSVLoader
import random

set_open_ai_api()
listings_loader = CSVLoader("listings.csv")
listing_documents=listings_loader.load()
print(f"There are {len(listing_documents)} listings loaded")
sample_listings = random.sample(listing_documents, 2)
print(f"Here are two sample loaded listings:\n{sample_listings}")

There are 28 listings loaded
Here are two sample loaded listings:
[Document(page_content='neighborhood: Sunset Hills\nprice: 550000\nbedrooms: 3\nbathrooms: 2\nhouse_size: 2200\ndescription: Modern ranch-style home with a sleek design and large windows that let in plenty of natural light. The backyard patio offers stunning views of the sunset over the rolling hills.\nneighborhood_description: Sunset Hills is a peaceful community with hiking trails and green spaces for outdoor enthusiasts. Conveniently located near shopping centers, restaurants, and top-rated schools, making it an ideal place for families.', metadata={'source': 'listings.csv', 'row': 1}), Document(page_content='neighborhood: Cedarwood Park\nprice: 225000\nbedrooms: 3\nbathrooms: 2\nhouse_size: 1600\ndescription: Cozy home with a spacious backyard and deck for outdoor gatherings and relaxation. The open floor plan and large windows create a bright and inviting atmosphere throughout the house.\nneighborhood_description: C

In [316]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings_model=OpenAIEmbeddings()
# I found no better way to ensure the chromadb is reset other than running twice with first call including delete_collection()
# Without these, each time the cell is rerun the documents are added all over new leading to wrong results in k-nearest similarity search
listings_db=Chroma.from_documents(documents=listing_documents,embedding=embeddings_model).delete_collection()
listings_db=Chroma.from_documents(documents=listing_documents,embedding=embeddings_model)

In [317]:
listings_db._collection.count()

28

In [318]:
most_similar_documents=listings_db.similarity_search(query=preference_summary_by_smart_assistant,k=4)
print(f"Most similar documents to the user preferences: {most_similar_documents}")

Most similar documents to the user preferences: [Document(page_content='neighborhood: Oakridge Park\nprice: 375000\nbedrooms: 3\nbathrooms: 2\nhouse_size: 1900\ndescription: Traditional home with a spacious backyard and deck for outdoor entertaining and relaxation. The cozy living room and updated kitchen provide a comfortable and inviting space for families to enjoy.\nneighborhood_description: Oakridge Park is a quiet and friendly neighborhood with well-maintained parks and playgrounds for children to play. Close to shopping centers, restaurants, and schools, making it a convenient and desirable place to live for families and young professionals.', metadata={'row': 10, 'source': 'listings.csv'}), Document(page_content='neighborhood: Oakwood Park\nprice: 400000\nbedrooms: 3\nbathrooms: 2\nhouse_size: 1800\ndescription: Traditional home with a spacious backyard and deck for outdoor entertaining and relaxation. The cozy living room and updated kitchen provide a comfortable and inviting s

In [319]:
most_similar_documents_string="\n\n---\n".join([most_similar_document.page_content for most_similar_document in most_similar_documents])
print(most_similar_documents_string)

neighborhood: Oakridge Park
price: 375000
bedrooms: 3
bathrooms: 2
house_size: 1900
description: Traditional home with a spacious backyard and deck for outdoor entertaining and relaxation. The cozy living room and updated kitchen provide a comfortable and inviting space for families to enjoy.
neighborhood_description: Oakridge Park is a quiet and friendly neighborhood with well-maintained parks and playgrounds for children to play. Close to shopping centers, restaurants, and schools, making it a convenient and desirable place to live for families and young professionals.

---
neighborhood: Oakwood Park
price: 400000
bedrooms: 3
bathrooms: 2
house_size: 1800
description: Traditional home with a spacious backyard and deck for outdoor entertaining and relaxation. The cozy living room and updated kitchen provide a comfortable and inviting space for families to enjoy.
neighborhood_description: Oakwood Park is a quiet and friendly neighborhood with well-maintained parks and playgrounds for c

Finally, we hit the LLM now with the extended context involving the most similar documents found from the vector db

In [320]:
response=qa_chain_with_listings(inputs={"question":"yes","listings":most_similar_documents_string})

In [321]:
print(f"The smart assistant answered: {response['answer']}")

The smart assistant answered: Based on your preferences, I have the following recommendations for you:

1. Oakridge Park
   - Price: $375,000
   - Bedrooms: 3
   - Bathrooms: 2
   - House Size: 1900 sqft
   - Description: Traditional home with a spacious backyard and deck for outdoor entertaining and relaxation. The cozy living room and updated kitchen provide a comfortable and inviting space for families to enjoy.
   - Neighborhood Description: Oakridge Park is a quiet and friendly neighborhood with well-maintained parks and playgrounds for children to play. Close to shopping centers, restaurants, and schools, making it a convenient and desirable place to live for families and young professionals.

2. Oakwood Park
   - Price: $400,000
   - Bedrooms: 3
   - Bathrooms: 2
   - House Size: 1800 sqft
   - Description: Traditional home with a spacious backyard and deck for outdoor entertaining and relaxation. The cozy living room and updated kitchen provide a comfortable and inviting space 

Let's verify if the top listing indeed exists

In [322]:
import pandas as pd
df=pd.read_csv("listings.csv")
filtered_df=df[df["neighborhood"]=="Oakridge Park"]
filtered_df.head()

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
10,Oakridge Park,375000,3,2,1900,Traditional home with a spacious backyard and ...,Oakridge Park is a quiet and friendly neighbor...


q.e.d. Good job :)

Step 6 - Personalized Response

The assignment requires that the LLM is used to give a response of suitable listings wherein the listing is described in such a way such that the aspects of relevance to the user are highlighted

In [360]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate


memory_with_listings=ExtendedConversationBufferMemory(
    conversation_history=create_chat_message_history(questions,answers),
    llm=llm,
    chat_memory=create_chat_message_history(questions,answers),
    memory_key="context",
    extra_variables=["listings"]) # resetting the memory with just the questions/answers

# We do the personalization of the listing response by instructing the LLM to do so
qa_template_with_listings_and_personalization=PromptTemplate(template=
(("""%s. VERY IMPORTANT:
- Only provide suggestions from the available listings.
- REPHRASE the listing description highlighting aspects of relevance to the user's preferences encoded under Question:'

Context: {context}

Listings: {listings}

Question: {question}
""") % base_system_prompt),
                                         input_variables=["context", "listings", "question"]
                                         )



qa_chain_with_listings_and_personalization = ConversationChain(
    llm=llm,
    memory=memory_with_listings,                                                   prompt=qa_template_with_listings_and_personalization,
    input_key="question",
    output_key="answer",
    verbose=False)



In [361]:
ask_llm_and_print_question_and_response(qa_chain_with_listings_and_personalization,question="I am waiting for your recommendations")

The user asked: I am waiting for your recommendations
The smart assistant answered: Are you ready to receive recommendations based on your preferences? If so, please type 'yes' or 'no'.


{'question': 'I am waiting for your recommendations',
 'context': 'AI: How big should your house be?\nHuman: A comfortable house with at least 3 bedrooms with a spacious kitchen and a cozy living room with at least 2000sqft\nAI: What are 3 most important things for you in choosing this property?\nHuman: A quiet neighborhood, good local schools, and convenient shopping options.\nAI: What is the maximum price you would be willing to pay?Which amenities would you like?\nHuman: $500,000\nAI: Which transportation options are important to you?\nHuman: A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.\nAI: How urban do you want your neighborhood to be?\nHuman: Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.',
 'listings': None,
 'answer': "Are you ready to receive recommendations based on your preferences? If so, please type 'yes' or 'no'."}

In [362]:
response=ask_llm_and_print_question_and_response(qa_chain_with_listings_and_personalization,question="yes")

The user asked: yes
The smart assistant answered: Based on your preferences, I understand that you are looking for a comfortable house with at least 3 bedrooms, a spacious kitchen, and a cozy living room with at least 2000sqft. You value a quiet neighborhood, good local schools, and convenient shopping options. Your maximum budget is $500,000 and you are looking for a property with a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Additionally, you prefer easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.

Are you ready to receive recommendations based on your preferences? If so, please type 'yes' or 'no'.


In [363]:
preference_summary_by_smart_assistant=response['answer']

In [364]:
preference_summary_by_smart_assistant

"Based on your preferences, I understand that you are looking for a comfortable house with at least 3 bedrooms, a spacious kitchen, and a cozy living room with at least 2000sqft. You value a quiet neighborhood, good local schools, and convenient shopping options. Your maximum budget is $500,000 and you are looking for a property with a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Additionally, you prefer easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.\n\nAre you ready to receive recommendations based on your preferences? If so, please type 'yes' or 'no'."

In [365]:
most_similar_documents=listings_db.similarity_search(query=preference_summary_by_smart_assistant,k=4)
most_similar_documents_string="\n\n---\n".join([most_similar_document.page_content for most_similar_document in most_similar_documents])

In [366]:
response=qa_chain_with_listings(inputs={"question":"yes","listings":most_similar_documents_string})
print(f"Here's the final response by the assistant which should have relevant listings rephrased to highlight relevant aspects of the description to the user's preferences:\n"
      f"{response['answer']}")

Here's the final response by the assistant which should have relevant listings rephrased to highlight relevant aspects of the description to the user's preferences:
Based on your preferences, I have the following recommendations for you:

1. Cedarwood Village
   - Price: $225,000
   - Bedrooms: 3
   - Bathrooms: 2
   - House Size: 1600 sqft
   - Description: Cozy home with a spacious backyard and deck for outdoor gatherings and relaxation. The open floor plan and large windows create a bright and inviting atmosphere throughout the house.
   - Neighborhood Description: Cedarwood Village is a family-friendly neighborhood with tree-lined streets and nearby parks for outdoor activities. Close to schools, shopping centers, and restaurants, making it a convenient and welcoming place to call home.

2. Hillside Village
   - Price: $200,000
   - Bedrooms: 2
   - Bathrooms: 1
   - House Size: 1600 sqft
   - Description: Charming cottage-style home with a cozy fireplace and a private backyard oas

In [367]:
import re

# Use regular expressions to extract the description of the top listing
match = re.search(r"1\.\s.*\n\s+-\sDescription:\s(.+?)\n\s+-\sNeighborhood\sDescription:", response["answer"], re.DOTALL)
if match:
    top_listing_description = match.group(1).strip()
    print(f"Description of the top listing:\n{top_listing_description}")
else:
    print("Top listing description not found.")

Description of the top listing:
Charming cottage-style home with a cozy fireplace and a private backyard oasis with a deck and garden. The bright and airy living spaces create a warm and inviting atmosphere for relaxing and entertaining.


Let's compare the original description with the one provided at the top result

In [368]:
assert len(df[df["neighborhood"]=="Cedarwood Village"]) == 1
df[df["neighborhood"]=="Oakridge Park"].iloc[0]["description"]

'Traditional home with a spacious backyard and deck for outdoor entertaining and relaxation. The cozy living room and updated kitchen provide a comfortable and inviting space for families to enjoy.'

Let's revisit the provided user answers about preferences and check that the rephrased listing highlights relevant aspects

In [370]:
for question, answer in zip(questions,answers):
    print(f"Q: {question}")
    print(f"a: {answer}")

Q: How big should your house be?
a: A comfortable house with at least 3 bedrooms with a spacious kitchen and a cozy living room with at least 2000sqft
Q: What are 3 most important things for you in choosing this property?
a: A quiet neighborhood, good local schools, and convenient shopping options.
Q: What is the maximum price you would be willing to pay?Which amenities would you like?
a: $500,000
Q: Which transportation options are important to you?
a: A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
Q: How urban do you want your neighborhood to be?
a: Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.
